In [0]:
import pandas as pd
import numpy as np
import string
import re
from unicodedata import normalize
from matplotlib import pyplot as plt
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint

Using TensorFlow backend.


In [0]:
df = pd.read_csv("/content/drive/My Drive/Deep Learning/deueng/deu.txt",encoding='utf-8', delimiter = r'\t', header=None)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: ParserWarning: Falling back to the 'python' engine because the 'c' engine does not support regex separators (separators > 1 char and different from '\s+' are interpreted as regex); you can avoid this warning by specifying engine='python'.
  """Entry point for launching an IPython kernel.


In [0]:
df.columns = ['English', 'German']

In [0]:
df['German'] = df['German'].str.normalize('NFKD').str.encode('ascii', errors='ignore').str.decode('utf-8')

In [0]:
df['English']  = df['English'].str.replace("won't","will not")
df['English']  = df['English'].str.replace("ain't","am not")
df['English']  = df['English'].str.replace("'s"," is")
df['English']  = df['English'].str.replace("'m"," am")
df['English']  = df['English'].str.replace("'re'"," are")
df['English']  = df['English'].str.replace("can't","can not")
df['English']  = df['English'].str.replace("'ll"," will")
df['English']  = df['English'].str.replace("n't"," not")

In [0]:
df['German'] = df['German'].str.replace("'s"," es")

In [0]:
df['English'] = df['English'].map(lambda x: x.lower())
df['German'] = df['German'].map(lambda x: x.lower())

In [0]:
df['English'] = df['English'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x)) #removing all charecters except alphabets
df['English'] = df['English'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
df['English'] = df['English'].map(lambda x: ' '.join(word for word in x.split() if len(word)>1)) #removing single charecters'

In [0]:
df['German'] = df['German'].apply(lambda x: re.sub('[^a-zA-z0-9\s]','',x)) #removing all charecters except alphabets
df['German'] = df['German'].apply(lambda x: re.sub(r'\s+',' ',x)) #removing extra spaces
df['German'] = df['German'].map(lambda x: ' '.join(word for word in x.split() if len(word)>1)) #removing single charecters'

In [0]:
df['English'][22519]

'call 110 right now'

In [0]:
df.shape

(192881, 2)

In [0]:
df = df[0:10000]

In [0]:
df_train = df[:9000]
df_test = df[9000:]

In [0]:
english_tokenizer = Tokenizer()
english_tokenizer.fit_on_texts(df['English'])
eng_vocab_size = len(english_tokenizer.word_index)+1
print (eng_vocab_size)
print (max([len(x) for x in df['English'].str.split()]))

2205
5


In [0]:
eng_max_len = (max([len(x) for x in df['English'].str.split()]))

In [0]:
german_tokenizer = Tokenizer()
german_tokenizer.fit_on_texts(df['German'])
german_vocab_size = len(german_tokenizer.word_index)+1
print (german_vocab_size)
print (max([len(x) for x in df['German'].str.split()]))

3565
10


In [0]:
german_max_len = (max([len(x) for x in df['German'].str.split()]))

In [0]:
trainX = german_tokenizer.texts_to_sequences(df_train['German'])
trainX = pad_sequences(trainX, maxlen=german_max_len, padding='post')

In [0]:
trainX.shape

(9000, 10)

In [0]:
trainY = english_tokenizer.texts_to_sequences(df_train['English'])
trainY = pad_sequences(trainY, maxlen=eng_max_len, padding='post')

In [0]:
trainY.shape

(9000, 5)

In [0]:
eng_max_len = (max([len(x) for x in df['English'].str.split()]))

In [0]:
trainY = to_categorical(trainY, num_classes=eng_vocab_size)

In [0]:
trainY.shape

(9000, 5, 2205)

In [0]:
testX = german_tokenizer.texts_to_sequences(df_test['German'])
testX = pad_sequences(testX, maxlen=german_max_len, padding='post')
testY = english_tokenizer.texts_to_sequences(df_test['English'])
testY = pad_sequences(testY, maxlen=eng_max_len, padding='post')
testY = to_categorical(testY, num_classes=eng_vocab_size)
testY.shape

(1000, 5, 2205)

In [0]:
embedding_size = 256

In [0]:
model = Sequential()
model.add(Embedding(german_vocab_size,embedding_size,input_length=german_max_len))
model.add(LSTM(256))
model.add(RepeatVector(eng_max_len))
model.add(LSTM(256,return_sequences=True))
model.add(TimeDistributed(Dense(eng_vocab_size,activation='softmax')))
model.compile(optimizer='adam', loss='categorical_crossentropy')
model.summary()

Instructions for updating:
Colocations handled automatically by placer.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 10, 256)           912640    
_________________________________________________________________
lstm_1 (LSTM)                (None, 256)               525312    
_________________________________________________________________
repeat_vector_1 (RepeatVecto (None, 5, 256)            0         
_________________________________________________________________
lstm_2 (LSTM)                (None, 5, 256)            525312    
_________________________________________________________________
time_distributed_1 (TimeDist (None, 5, 2205)           566685    
Total params: 2,529,949
Trainable params: 2,529,949
Non-trainable params: 0
_________________________________________________________________


In [0]:
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')

In [0]:
model.fit(trainX, trainY, epochs=30, batch_size=64, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

Instructions for updating:
Use tf.cast instead.
Train on 9000 samples, validate on 1000 samples
Epoch 1/30
 - 8s - loss: 3.8122 - val_loss: 4.3183

Epoch 00001: val_loss improved from inf to 4.31826, saving model to model.h5
Epoch 2/30
 - 5s - loss: 3.2471 - val_loss: 4.2471

Epoch 00002: val_loss improved from 4.31826 to 4.24711, saving model to model.h5
Epoch 3/30
 - 4s - loss: 3.1870 - val_loss: 4.2517

Epoch 00003: val_loss did not improve from 4.24711
Epoch 4/30
 - 4s - loss: 3.1581 - val_loss: 4.2506

Epoch 00004: val_loss did not improve from 4.24711
Epoch 5/30
 - 4s - loss: 3.1415 - val_loss: 4.2943

Epoch 00005: val_loss did not improve from 4.24711
Epoch 6/30
 - 4s - loss: 3.1303 - val_loss: 4.2643

Epoch 00006: val_loss did not improve from 4.24711
Epoch 7/30
 - 4s - loss: 3.1223 - val_loss: 4.2813

Epoch 00007: val_loss did not improve from 4.24711
Epoch 8/30
 - 4s - loss: 3.1081 - val_loss: 4.2013

Epoch 00008: val_loss improved from 4.24711 to 4.20132, saving model to mod

In [0]:
englishword_for_id = dict((v,k) for k,v in english_tokenizer.word_index.items())  
englishword_for_id[1]

'tom'

In [0]:
for word in np.argmax(model.predict(testX[4:5]),axis=2)[0]:
  if word != 0:
    print (englishword_for_id[word])

you
must
go


In [0]:
pad_sequences(german_tokenizer.texts_to_sequences(["Guten Morgen Jagan"]),padding="post",maxlen=10)

array([[1165,  424,    0,    0,    0,    0,    0,    0,    0,    0]],
      dtype=int32)

In [0]:
predict_sentence_split = []
for word in np.argmax(model.predict(pad_sequences(german_tokenizer.texts_to_sequences([df['German'][9999]]),padding="post",maxlen=10)),axis=2)[0]:
  if word != 0:
    print (englishword_for_id[word])
    predict_sentence_split.append(englishword_for_id[word])

found
him


In [0]:
from nltk.translate.bleu_score import sentence_bleu

In [0]:
df['English'][9999]

'just found it'

In [0]:
sentence_bleu([df['English'][9999].split()],predict_sentence_split)

/usr/local/lib/python3.6/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.510029457493824